In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Get the stock price, pe ratio, pb ratio, industry and dividend yield of the stock

def get_stock_data(stock):
    stock_data = yf.Ticker(stock)
    stock_info = stock_data.info
    
    # Retrieve stock data with fallback to NaN
    stock_price = stock_info.get('regularMarketPreviousClose', np.nan)  # Default to NaN if missing
    
    # Check for 'trailingPE', then 'forwardPE', and fallback to NaN
    pe_ratio = stock_info.get('trailingPE') if stock_info.get('trailingPE') is not None else stock_info.get('forwardPE', np.nan)
    
    pb_ratio = stock_info.get('priceToBook', np.nan)  # Default to NaN if missing
    
    # Handle dividend yield (convert to percentage if available)
    dividend_yield = stock_info.get('dividendYield', 0)  # Default to 0 if missing
    if dividend_yield is not None:
        dividend_yield *= 100
    
    # Default industry to 'Unknown' if missing
    industry = stock_info.get('industry', 'Unknown')
    
    #Get the mean recommendation of the stock
    mean_recommendation = stock_info.get('targetMeanPrice', np.nan)
    #Get the median recommendation of the stock
    median_recommendation = stock_info.get('targetMedianPrice', np.nan)
    # calculate financial intrinsic value of the stock using the financials Intrinsic value = Earnings per share (EPS) x (1 + r) x P/E ratio
    # where r is the expected growth rate of the company
    # calculate the growth rate of the company
    growth_rate = stock_info.get('earningsGrowth', np.nan)
    # calculate the earnings per share
    eps = stock_info.get('trailingEps', np.nan)
    # calculate the financial intrinsic value, if growth rate and eps is not available, set intrinsic value to NaN
    if growth_rate is not None and eps is not None:
        financial_intrinsic_value = eps * (1 + growth_rate) * pe_ratio
    else:
        financial_intrinsic_value = np.nan
    
    
    return stock_price, pe_ratio, pb_ratio, dividend_yield, mean_recommendation, median_recommendation, financial_intrinsic_value, industry


In [ ]:
stock_data = yf.Ticker('AAPL')
stock_data.info

#get all names of columns
stock_data.info.keys()

dict_keys(['address1', 'city', 'state', 'zip', 'country', 'phone', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'governanceEpochDate', 'compensationAsOfEpochDate', 'irWebsite', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currenc

In [ ]:
import requests


def send_telegram_message(message):
    bot_token = "INSERT YOURS HERE"
    chat_id = "INSERT YOURS HERE"
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"

    params = {"chat_id": chat_id, "text": message}

    response = requests.post(url, params=params)
    if response.status_code == 200:
        print("Message sent successfully!")
    else:
        print(f"Failed to send message. Status code: {response.status_code}")

In [3]:
from bs4 import BeautifulSoup
import requests
import numpy as np

url = 'http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/pedata.html'

def get_pe_averages():
    # Disable SSL verification
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find_all('table')[0]  # Find the first table
    rows = table.find_all('tr')
    pe_averages = {}
    for row in rows[1:]:  # Skip the header row
        items = row.find_all('td')
        industry = items[0].text.strip()
        pe = items[3].text.strip()
        pe_averages[industry] = pe
    return pe_averages

def get_pe_average(industry, pe_averages):
    return pe_averages.get(industry, np.nan)

# Get the PE averages
pe_averages = get_pe_averages()
print(get_pe_average('Advertising', pe_averages))


/opt/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pages.stern.nyu.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


54.85


In [4]:
#get the pb ratios of the industries
url = 'https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/pbvdata.html'
def get_pb_averages():
    # Disable SSL verification
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find_all('table')[0]  # Find the first table
    rows = table.find_all('tr')
    pb_averages = {}
    for row in rows[1:]:  # Skip the header row
        items = row.find_all('td')
        industry = items[0].text.strip()
        pb = items[2].text.strip()
        pb_averages[industry] = pb
    return pb_averages

def get_pb_average(industry, pb_averages):
    return pb_averages.get(industry, np.nan)


#try to get the pb ratio of the industry
pb_averages = get_pb_averages()
print(get_pb_average('Advertising', pb_averages))

/opt/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pages.stern.nyu.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


5.76


In [5]:
#check the keys in pe_averages
pe_averages.keys()

dict_keys(['Advertising', 'Aerospace/Defense', 'Air Transport', 'Apparel', 'Auto &\n\t\t        Truck', 'Auto Parts', 'Bank (Money\n\t\t        Center)', 'Banks (Regional)', 'Beverage\n\t\t        (Alcoholic)', 'Beverage (Soft)', 'Broadcasting', 'Brokerage & Investment Banking', 'Building\n\t\t        Materials', 'Business & Consumer Services', 'Cable TV', 'Chemical (Basic)', 'Chemical\n\t\t        (Diversified)', 'Chemical (Specialty)', 'Coal &\n\t\t        Related Energy', 'Computer Services', 'Computers/Peripherals', 'Construction Supplies', 'Diversified', 'Drugs (Biotechnology)', 'Drugs\n\t\t        (Pharmaceutical)', 'Education', 'Electrical\n\t\t        Equipment', 'Electronics (Consumer & Office)', 'Electronics\n\t\t        (General)', 'Engineering/Construction', 'Entertainment', 'Environmental & Waste Services', 'Farming/Agriculture', 'Financial Svcs. (Non-bank & Insurance)', 'Food Processing', 'Food Wholesalers', 'Furn/Home\n\t\t        Furnishings', 'Green & Renewable Energy'

In [6]:
import re

# Clean the keys in pe_averages using regex
pe_averages = {re.sub(r'\s+', ' ', re.sub(r'\n\t\t', '', industry)).strip(): pe for industry, pe in pe_averages.items()}
print(pe_averages)

{'Advertising': '54.85', 'Aerospace/Defense': '36.96', 'Air Transport': '2426.56', 'Apparel': '23.77', 'Auto & Truck': '20.30', 'Auto Parts': '31.19', 'Bank (Money Center)': '9.95', 'Banks (Regional)': '25.48', 'Beverage (Alcoholic)': '49.69', 'Beverage (Soft)': '57.10', 'Broadcasting': '13.11', 'Brokerage & Investment Banking': '21.55', 'Building Materials': '25.93', 'Business & Consumer Services': '541.75', 'Cable TV': '12.56', 'Chemical (Basic)': '9.68', 'Chemical (Diversified)': '6.88', 'Chemical (Specialty)': '19.14', 'Coal & Related Energy': '9.07', 'Computer Services': '37.48', 'Computers/Peripherals': '38.84', 'Construction Supplies': '31.78', 'Diversified': '118.80', 'Drugs (Biotechnology)': '3263.83', 'Drugs (Pharmaceutical)': '38.94', 'Education': '30.36', 'Electrical Equipment': '47.28', 'Electronics (Consumer & Office)': '106.94', 'Electronics (General)': '30.07', 'Engineering/Construction': '44.68', 'Entertainment': '36.47', 'Environmental & Waste Services': '39.67', 'Far

In [ ]:
pb_averages = {re.sub(r'\s+', ' ', re.sub(r'\n\t\t', '', industry)).strip(): pb for industry, pb in pb_averages.items()}

In [164]:
print(pe_averages)

{'Advertising': '54.85', 'Aerospace/Defense': '36.96', 'Air Transport': '2426.56', 'Apparel': '23.77', 'Auto & Truck': '20.30', 'Auto Parts': '31.19', 'Bank (Money Center)': '9.95', 'Banks (Regional)': '25.48', 'Beverage (Alcoholic)': '49.69', 'Beverage (Soft)': '57.10', 'Broadcasting': '13.11', 'Brokerage & Investment Banking': '21.55', 'Building Materials': '25.93', 'Business & Consumer Services': '541.75', 'Cable TV': '12.56', 'Chemical (Basic)': '9.68', 'Chemical (Diversified)': '6.88', 'Chemical (Specialty)': '19.14', 'Coal & Related Energy': '9.07', 'Computer Services': '37.48', 'Computers/Peripherals': '38.84', 'Construction Supplies': '31.78', 'Diversified': '118.80', 'Drugs (Biotechnology)': '3263.83', 'Drugs (Pharmaceutical)': '38.94', 'Education': '30.36', 'Electrical Equipment': '47.28', 'Electronics (Consumer & Office)': '106.94', 'Electronics (General)': '30.07', 'Engineering/Construction': '44.68', 'Entertainment': '36.47', 'Environmental & Waste Services': '39.67', 'Far

In [165]:
print(pb_averages)

{'Advertising': '5.76', 'Aerospace/Defense': '5.08', 'Air Transport': '2.24', 'Apparel': '2.56', 'Auto & Truck': '4.60', 'Auto Parts': '2.00', 'Bank (Money Center)': '1.05', 'Banks (Regional)': '1.02', 'Beverage (Alcoholic)': '3.10', 'Beverage (Soft)': '7.42', 'Broadcasting': '0.80', 'Brokerage & Investment Banking': '1.66', 'Building Materials': '4.74', 'Business & Consumer Services': '5.54', 'Cable TV': '2.06', 'Chemical (Basic)': '1.98', 'Chemical (Diversified)': '1.97', 'Chemical (Specialty)': '2.67', 'Coal & Related Energy': '1.79', 'Computer Services': '4.37', 'Computers/Peripherals': '30.98', 'Construction Supplies': '3.92', 'Diversified': '1.87', 'Drugs (Biotechnology)': '6.14', 'Drugs (Pharmaceutical)': '5.20', 'Education': '2.72', 'Electrical Equipment': '2.92', 'Electronics (Consumer & Office)': '1.90', 'Electronics (General)': '3.28', 'Engineering/Construction': '3.33', 'Entertainment': '2.59', 'Environmental & Waste Services': '6.25', 'Farming/Agriculture': '2.73', 'Financ

In [10]:
# List of stocks to analyze
stocks = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'HSY', 'KO', 'PEP', 'NKE', 'V', 'INTC', 'NVDA', 'AMD', 'IBM', 'ORCL', 'ASML']

# Get information for each stock
results = []
for stock in stocks:
    stock_price, pe_ratio, pb_ratio, dividend_yield, mean_rec, median_rec, financial_iv, industry = get_stock_data(stock)
    if industry == 'Semiconductors' or industry == 'Semiconductor Equipment & Materials':
        industry = 'Semiconductor'
    #map Software-Infrastructure to Software (System & Application)
    if industry == 'Software - Infrastructure' or industry == 'Information Technology Services':
        industry = 'Software (System & Application)'
    #map Consumer Electronics to Software (Entertainment)
    if industry == 'Consumer Electronics':
        industry = 'Software (Entertainment)'
    #map Internet Content & Information to Software (Internet)
    if industry == 'Internet Content & Information' or industry == 'Internet Retail':
        industry = 'Software (Internet)'
    #map Confectioners to Food Processing
    if industry == 'Confectioners':
        industry = 'Food Processing'
    #map Beverages - Non-Alcoholic to Beverages (Soft)
    if industry == 'Beverages - Non-Alcoholic':
        industry = 'Beverage (Soft)'
    #map Credit Services to Financial Svcs. (Non-bank & Insurance)	
    if industry == 'Credit Services':
        industry = 'Financial Svcs. (Non-bank & Insurance)'
    #map Footwear & Accessories to Apparel
    if industry == 'Footwear & Accessories':
        industry = 'Apparel'
    pe_average = get_pe_average(industry, pe_averages)
    pb_average = get_pb_average(industry, pb_averages)
    results.append([stock, industry, stock_price, pe_ratio, pb_ratio, dividend_yield, pe_average, pb_average, mean_rec, median_rec, financial_iv])


In [11]:
# Create a DataFrame from the results
columns = ['Stock', 'Industry', 'Price', 'PE Ratio', 'PB Ratio', 'Dividend Yield', 'Industry PE', 'Industry PB', 'Target Mean Price', 'Target Median Price', 'Financial Intrinsic Value']
df = pd.DataFrame(results, columns=columns)


In [12]:
df

,Stock,Industry,Price,PE Ratio,PB Ratio,Dividend Yield,Industry PE,Industry PB,Target Mean Price,Target Median Price,Financial Intrinsic Value
0,AAPL,Software (Entertainment),228.52,37.869850,61.022034,0.44,149.98,NaN,244.47739,250.000,151.484323
1,MSFT,Software (System & Application),412.87,34.491314,10.777143,0.80,133.38,NaN,504.76813,500.000,460.367985
2,AMZN,Software (Internet),198.38,42.209846,7.995132,0.00,28.53,9.00,234.13594,234.500,299.819491
3,GOOGL,Software (Internet),167.63,21.822515,6.432671,0.49,28.53,9.00,209.64458,211.000,225.062144
4,META,Software (Internet),563.09,26.386974,8.577609,0.36,28.53,9.00,648.35046,650.000,768.258331
5,HSY,Food Processing,174.95,20.070034,8.412011,3.13,49.53,2.30,183.65360,180.000,152.609127
6,KO,Beverage (Soft),63.76,26.522820,10.388429,3.04,57.10,NaN,74.38458,75.500,59.445596
7,PEP,Beverage (Soft),160.34,23.858616,11.434217,3.35,57.10,NaN,181.93571,183.000,154.062003
8,NKE,Apparel,75.10,22.177650,8.276304,2.07,23.77,2.56,91.24314,92.000,57.662999
9,V,Financial Svcs. (Non-bank & Insurance),309.90,31.852007,15.436570,0.76,34.63,NaN,321.03200,325.000,362.296513


In [170]:
for index, row in df.iterrows():
    stock = row['Stock']
    industry = row['Industry']
    price = row['Price']
    pe_ratio = row['PE Ratio']
    pb_ratio = row['PB Ratio']
    div_yield = row['Dividend Yield']
    pe_avg = float(row['Industry PE']) if not pd.isna(row['Industry PE']) else float('nan')  # Convert to float
    pb_avg = float(row['Industry PB']) if not pd.isna(row['Industry PB']) else float('nan')  # Convert to float
    target_mean_price = row['Target Mean Price']
    target_median_price = row['Target Median Price']
    intrinsic_val = row['Financial Intrinsic Value']
    
    # Format the message
    message = f"Stock: {stock}\n"
    message += f"Industry: {industry}\n"
    message += f"Price: ${price:.2f}\n"
    message += f"PE Ratio: {pe_ratio:.2f} (Industry Avg: {pe_avg:.2f})\n"
    message += f"PB Ratio: {pb_ratio:.2f} (Industry Avg: {pb_avg:.2f})\n"
    message += f"Dividend Yield: {div_yield:.2f}%\n"
    message += f"Target Mean Price: ${target_mean_price:.2f}\n"
    message += f"Target Median Price: ${target_median_price:.2f}\n"
    message += f"Financial Intrinsic Value: ${intrinsic_val:.2f}\n"
    
    # Recommendation logic
    if pe_ratio < pe_avg and pb_ratio < pb_avg:
        message += "Recommendation: Buy\n"
    elif pe_ratio > pe_avg and pb_ratio > pb_avg:
        message += "Recommendation: Sell\n"
    else:
        message += "Recommendation: Hold\n"
    
    # Send to Telegram
    send_telegram_message(message)
    
    print(message)


Message sent successfully!
Stock: AAPL
Industry: Software (Entertainment)
Price: $229.00
PE Ratio: 37.52 (Industry Avg: 149.98)
PB Ratio: 60.66 (Industry Avg: 6.24)
Dividend Yield: 0.44%
Target Mean Price: $244.48
Target Median Price: $250.00
Financial Intrinsic Value: $150.59
Recommendation: Hold

Message sent successfully!
Stock: MSFT
Industry: Software (System & Application)
Price: $414.66
PE Ratio: 34.15 (Industry Avg: 133.38)
PB Ratio: 10.67 (Industry Avg: 11.03)
Dividend Yield: 0.80%
Target Mean Price: $504.77
Target Median Price: $500.00
Financial Intrinsic Value: $455.81
Recommendation: Buy

Message sent successfully!
Stock: AMZN
Industry: Software (Internet)
Price: $202.88
PE Ratio: 42.30 (Industry Avg: 28.53)
PB Ratio: 8.05 (Industry Avg: 9.00)
Dividend Yield: 0.00%
Target Mean Price: $234.14
Target Median Price: $234.50
Financial Intrinsic Value: $301.74
Recommendation: Hold

Message sent successfully!
Stock: GOOGL
Industry: Software (Internet)
Price: $175.98
PE Ratio: 22.26

In [ ]:
# send a message on how financial intrinsic value is calculated
message = "The financial intrinsic value is calculated using the formula: Earnings per share (EPS) x (1 + r) x P/E ratio\n"
message += "where r is the expected growth rate of the company\n"
message += "If the growth rate and EPS are not available, the intrinsic value is set to NaN"
send_telegram_message(message)